# <b> Calculating statistics for annual and seasonal runoff

### Updated 4/17 by MA

## <b> Setting up the workspace

In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import shapely.geometry as shpg

In [2]:
from oggm import cfg, utils, workflow, tasks, graphics

In [3]:
# Download glacier outlines
utils.get_rgi_dir(version='62')  # path to the data after download

'/home/ulteelab/OGGM/rgi/RGIV62'

In [4]:
# Get rgi region file for region 16 (South America)
fr = utils.get_rgi_region_file(16, version='62')

In [5]:
# Read file as as geopandas
gdf = gpd.read_file(fr)

In [6]:
# Add basin shapefile
path = '~/Documents/Molly-thesis/Data/Basin_shapefiles/output/LaPaz_dissolved.shp' 
basin = gpd.read_file(path)

In [7]:
# Select glaciers within the basin
in_bas = [basin.geometry.contains(shpg.Point(x, y))[0] for
          (x, y) in zip(gdf.CenLon, gdf.CenLat)]
gdf_sel = gdf.loc[in_bas]

In [8]:
# Select rgi id's from the gdf_sel file
rgi_ids = gdf_sel['RGIId']

## <b> Defining Functions

### Annual

In [9]:
def make_summed_series(fpath_stem, which_date, which_gcm, which_rcp):
    this_fpath = fpath_stem+'{}-gcm_data_{}_{}_compiled_output.nc'.format(which_date, which_gcm, which_rcp)
    this_ds = xr.open_dataset(this_fpath)
    melt1 = this_ds.melt_on_glacier
    melt2 = this_ds.melt_off_glacier
    melt3 = this_ds.liq_prcp_off_glacier
    melt4 = this_ds.liq_prcp_on_glacier
    
    this_sum = melt1.sum(dim='rgi_id').dropna(dim='time') + melt2.sum(dim='rgi_id').dropna(dim='time') + melt3.sum(dim='rgi_id').dropna(dim='time') + melt4.sum(dim='rgi_id').dropna(dim='time')
    nonzero_sum = this_sum.where(this_sum!=0, drop=True)
    nonzero_sum *= 1e-9
    return nonzero_sum

In [10]:
# Function for good gcms
def process_all_rcps(which_gcm, fpath_stem, which_date, rcp_list=['rcp26', 'rcp45', 'rcp60', 'rcp85']):
    gcm = which_gcm
    base_fpath = fpath_stem
    date = which_date
    all_series = []
    for rcp in rcp_list:
        s = make_summed_series(which_gcm=gcm, which_rcp=rcp, fpath_stem=base_fpath, which_date=date)
        all_series.append(s)
    
    return all_series

In [11]:
def process_3_rcps(which_gcm, fpath_stem, which_date, rcp_list=['rcp26', 'rcp45', 'rcp85']):
    gcm = which_gcm
    base_fpath = fpath_stem
    date = which_date
    all_series = []
    for rcp in rcp_list:
        s = make_summed_series(which_gcm=gcm, which_rcp=rcp, fpath_stem=base_fpath, which_date=date)
        all_series.append(s)
    
    return all_series

In [12]:
# Make a series with different runoff stems
def make_series(runoff_stem, fpath_stem, which_date, which_gcm, which_rcp):
    this_fpath = fpath_stem+'{}-gcm_data_{}_{}_compiled_output.nc'.format(which_date, which_gcm, which_rcp)
    this_ds = xr.open_dataset(this_fpath)
    this_ds_runoff = this_ds[runoff_stem]
    this_runoff = this_ds_runoff.sum(dim='rgi_id').dropna(dim='time')
    nonzero_sum = this_runoff.where(this_runoff!=0, drop=True)
    nonzero_sum *= 1e-9
    return nonzero_sum

In [13]:
# Make a dictionary for each runoff type
def to_dict(gcm_list, runoff_stem, fpath_stem, which_date, which_rcp):
    x=0
    dictionary = {}
    for gcm in gcm_list:
        rcp_values = make_series(runoff_stem, fpath_stem, which_date[x], which_gcm=gcm, which_rcp=rcp)
        k = '{}'.format(gcm)
        dictionary[k]=rcp_values
        x += 1
    return dictionary

In [72]:
# Turn each runoff type into a dataframe and find the mean
def to_df(dataframe):
    my_df = pd.DataFrame(dataframe)
    my_df['mean'] = my_df.transpose().mean(axis=0)
    return my_df

### Seasonal

In [14]:
# Function: load the dataset
def open_series(fpath_stem, which_date, which_gcm, which_rcp):
    this_fpath = fpath_stem+'{}-gcm_data_{}_{}_compiled_output.nc'.format(which_date, which_gcm, which_rcp)
    this_ds = xr.open_dataset(this_fpath)
    return this_ds

In [15]:
# Function: calculate runoff
def calc_seasonal(this_ds):
    melt1 = this_ds.melt_on_glacier_monthly
    melt2 = this_ds.melt_off_glacier_monthly
    melt3 = this_ds.liq_prcp_off_glacier_monthly
    melt4 = this_ds.liq_prcp_on_glacier_monthly
    
    this_sum = melt1.sum(dim='rgi_id').dropna(dim='time') + melt2.sum(dim='rgi_id').dropna(dim='time') + melt3.sum(dim='rgi_id').dropna(dim='time') + melt4.sum(dim='rgi_id').dropna(dim='time')
    nonzero_sum = this_sum.where(this_sum!=0, drop=True)
    nonzero_sum *= 1e-9

    return nonzero_sum

In [16]:
# Function: select a year and take the mean runoff of all gcms for this year
def season_sliced(keys, dictionary, time):
    new_df = pd.DataFrame()
    for k in keys:
        new = dictionary[k]
        new = new[time]
        new = pd.DataFrame(new)
        new_df[k] = new
    new_df['mean'] = new_df.transpose().mean()
    return new_df

In [17]:
# Make a function that calculates a given runoff value, which I can visualize later
def rtype_calc_monthly(keys, dictionary, runoff):
    dataframe = pd.DataFrame()
    for k in keys:
        rtype_gcm = dictionary[k]
        rtype = rtype_gcm[runoff]
        rtype = pd.DataFrame(rtype)
        dataframe[k] = rtype
    dataframe['mean'] = dataframe.transpose().mean()
    return dataframe

In [18]:
# Make a function that calculates a given runoff value, which I can visualize later
def rtype_calc_monthly_2vars(keys, dictionary, runoff1, runoff2):
    dataframe = pd.DataFrame()
    for k in keys:
        rtype_gcm = dictionary[k]
        rtype1 = rtype_gcm[runoff1]
        rtype2 = rtype_gcm[runoff2]
        rsum = rtype1 + rtype2
        rtype_sum = pd.DataFrame(rsum)
        dataframe[k] = rtype_sum
    dataframe['mean'] = dataframe.transpose().mean()
    return dataframe

## Things I want to do: <br>
- (DONE) Find values of volume at the beginning of the simulation and the end (for each RCP) and find the diff between RCP 26 and 85 <br>
- Find the spread of GCM values of volume for each rcp scenario (what is the range > max - min) <br>
- Find average range of GCM spread of total runoff during first decade and last decade of the simulation, also spread of each component <br>
- Find mean Month during which mean runoff is at it's peak and trough for each time slice (by RCP), also find spread and VALUE of runoff at that time <br>
- Find mean month of liquid precip vs melt off glacier vs melt on glacier peaks, also GCM spread <br>
- 

### Volume

In [19]:
gcm_list = ['CCSM4', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'CanESM2', 'GFDL-CM3', 'GFDL-ESM2G', 'GISS-E2-R', 'IPSL-CM5A-LR', 'MPI-ESM-LR', 'NorESM1-M']
date_list = ['2023-01-12', '2023-01-23', '2023-01-17', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-25', '2023-01-25', '2023-01-25']
base = '~/Documents/Molly-thesis/Data/'
rcp='rcp26'

runoff = 'volume'
rcp26_volume = to_dict(gcm_list, runoff_stem=runoff, fpath_stem=base, which_date=date_list, which_rcp=rcp)

rcp26_df_v = pd.DataFrame(rcp26_volume)

rcp26_df_v['mean'] = rcp26_df_v.transpose().mean()

In [20]:
gcm_list = ['CCSM4', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'CanESM2', 'GFDL-CM3', 'GFDL-ESM2G', 'GISS-E2-R', 'IPSL-CM5A-LR', 'MPI-ESM-LR', 'NorESM1-M']
date_list = ['2023-01-12', '2023-01-23', '2023-01-17', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-25', '2023-01-25', '2023-01-25']
base = '~/Documents/Molly-thesis/Data/'
rcp='rcp45'
runoff = 'volume'

rcp45_volume = to_dict(gcm_list, runoff_stem=runoff, fpath_stem=base, which_date=date_list, which_rcp=rcp)

rcp45_df_v = pd.DataFrame(rcp45_volume)

rcp45_df_v['mean'] = rcp45_df_v.transpose().mean()

In [21]:
gcm_list = ['CCSM4', 'CSIRO-Mk3-6-0', 'GFDL-CM3', 'GFDL-ESM2G', 'GISS-E2-R', 'IPSL-CM5A-LR', 'NorESM1-M']
date_list = ['2023-01-12', '2023-01-17', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-25', '2023-01-25']
base = '~/Documents/Molly-thesis/Data/'
rcp='rcp60'
runoff = 'volume'

rcp60_volume = to_dict(gcm_list, runoff_stem=runoff, fpath_stem=base, which_date=date_list, which_rcp=rcp)

rcp60_df_v = pd.DataFrame(rcp60_volume)

rcp60_df_v['mean'] = rcp60_df_v.transpose().mean()

In [22]:
gcm_list = ['CCSM4', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'CanESM2', 'GFDL-CM3', 'GFDL-ESM2G', 'GISS-E2-R', 'IPSL-CM5A-LR', 'MPI-ESM-LR', 'NorESM1-M']
date_list = ['2023-01-12', '2023-01-23', '2023-01-17', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-25', '2023-01-25', '2023-01-25']
base = '~/Documents/Molly-thesis/Data/'
rcp='rcp85'
runoff = 'volume'

rcp85_volume = to_dict(gcm_list, runoff_stem=runoff, fpath_stem=base, which_date=date_list, which_rcp=rcp)

rcp85_df_v = pd.DataFrame(rcp85_volume)

rcp85_df_v['mean'] = rcp85_df_v.transpose().mean()
#rcp_mean = rcp26_df.transpose().rolling(window=4).mean()

Volume loss (mean) per RCP scenario from time 0 to 80

In [23]:
def volume_loss(volume_df):
    vlost = volume_df.iloc[0]['mean'] - volume_df.iloc[80]['mean']
    return vlost

In [25]:
volume_loss(rcp26_df_v)

0.30431378

In [24]:
volume_loss(rcp85_df_v)

0.3458913

Volume differences at time 80 between 26 and 85

In [71]:
rcp26_df_v.iloc[80]['mean'] - rcp85_df_v.iloc[80]['mean']

0.041577518

Percent change volume per rcp

In [79]:
(volume_loss(rcp85_df_v)/(rcp85_df_v.iloc[0]['mean']))*100

65.76952934265137

Additional percent loss, 85 vs 26

In [66]:
(rcp85_df_v.iloc[80]['mean']-rcp26_df_v.iloc[80]['mean'])/(rcp26_df_v.iloc[0]['mean']-rcp26_df_v.iloc[80]['mean'])*100

-13.662712275981903

GCM spread - volume

In [45]:
rcp26_df_v.iloc[80].drop('mean').max() - rcp26_df_v.iloc[80].drop('mean').min()

0.03929542

In [40]:
rcp45_df_v.iloc[80].drop('mean').max() - rcp45_df_v.iloc[80].drop('mean').min()

0.027026176

In [41]:
rcp60_df_v.iloc[80].drop('mean').max() - rcp60_df_v.iloc[80].drop('mean').min()

0.027324528

In [42]:
rcp85_df_v.iloc[80].drop('mean').max() - rcp85_df_v.iloc[80].drop('mean').min()

0.029659897

### Total runoff

In [48]:
# Make a dictionary for rcp26 values
rcp26_dict = {}

gcm_list = ['CCSM4', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'CanESM2', 'GFDL-CM3', 'GFDL-ESM2G', 'GISS-E2-R', 'IPSL-CM5A-LR', 'MPI-ESM-LR', 'NorESM1-M']
date_list = ['2023-01-12', '2023-01-23', '2023-01-17', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-25', '2023-01-25', '2023-01-25']

base = '~/Documents/Molly-thesis/Data/'
rcp='rcp26'

x=0

for gcm in gcm_list:
    rcp26_values = make_summed_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=rcp)
    k = '{}'.format(gcm)
    rcp26_dict[k]=rcp26_values
    x += 1

rcp26_df = pd.DataFrame(rcp26_dict)

rcp26_df['mean'] = rcp26_df.transpose().mean()

In [49]:
# Make a dictionary for rcp45 values
rcp45_dict = {}

gcm_list = ['CCSM4', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'CanESM2', 'GFDL-CM3', 'GFDL-ESM2G', 'GISS-E2-R', 'IPSL-CM5A-LR', 'MPI-ESM-LR', 'NorESM1-M']
date_list = ['2023-01-12', '2023-01-23', '2023-01-17', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-25', '2023-01-25', '2023-01-25']

base = '~/Documents/Molly-thesis/Data/'
rcp='rcp45'

x=0

for gcm in gcm_list:
    rcp45_values = make_summed_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=rcp)
    k = '{}'.format(gcm)
    rcp45_dict[k]=rcp45_values
    x += 1

rcp45_df = pd.DataFrame(rcp45_dict)

rcp45_df['mean'] = rcp45_df.transpose().mean()
#rcp_mean = rcp26_df.transpose().rolling(window=4).mean()

In [50]:
# Make a dictionary for rcp60 values
rcp60_dict = {}

gcm_list = ['CCSM4', 'CSIRO-Mk3-6-0', 'GFDL-CM3', 'GFDL-ESM2G', 'GISS-E2-R', 'IPSL-CM5A-LR', 'NorESM1-M']
date_list = ['2023-01-12', '2023-01-17', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-25', '2023-01-25']

base = '~/Documents/Molly-thesis/Data/'
rcp='rcp60'

x=0

for gcm in gcm_list:
    rcp60_values = make_summed_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=rcp)
    k = '{}'.format(gcm)
    rcp60_dict[k]=rcp60_values
    x += 1
    
rcp60_df = pd.DataFrame(rcp60_dict)

rcp60_df['mean'] = rcp60_df.transpose().mean()

In [51]:
# Make a dictionary for rcp85 values
rcp85_dict = {}

gcm_list = ['CCSM4', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'CanESM2', 'GFDL-CM3', 'GFDL-ESM2G', 'GISS-E2-R', 'IPSL-CM5A-LR', 'MPI-ESM-LR', 'NorESM1-M']
date_list = ['2023-01-12', '2023-01-23', '2023-01-17', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-25', '2023-01-25', '2023-01-25']

base = '~/Documents/Molly-thesis/Data/'
rcp='rcp85'

x=0

for gcm in gcm_list:
    rcp85_values = make_summed_series(fpath_stem=base, which_date=date_list[x], which_gcm=gcm, which_rcp=rcp)
    k = '{}'.format(gcm)
    rcp85_dict[k]=rcp85_values
    x += 1

rcp85_df = pd.DataFrame(rcp85_dict)

rcp85_df['mean'] = rcp85_df.transpose().mean()

Calculate RCP spread

In [62]:
rcp26_df.iloc[80]['mean'] - rcp85_df.iloc[80]['mean']

0.6847992

GCM spread - total runoff

In [67]:
rcp26_df.iloc[80].drop('mean').max() - rcp26_df.iloc[80].drop('mean').min()

10.592567

In [68]:
rcp45_df.iloc[80].drop('mean').max() - rcp45_df.iloc[80].drop('mean').min()

19.804094

In [69]:
rcp60_df.iloc[80].drop('mean').max() - rcp60_df.iloc[80].drop('mean').min()

16.008806

In [70]:
rcp85_df.iloc[80].drop('mean').max() - rcp85_df.iloc[80].drop('mean').min()

27.447378

### Runoff by Component

In [73]:
gcm_list = ['CCSM4', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'CanESM2', 'GFDL-CM3', 'GFDL-ESM2G', 'GISS-E2-R', 'IPSL-CM5A-LR', 'MPI-ESM-LR', 'NorESM1-M']
date_list = ['2023-01-12', '2023-01-23', '2023-01-17', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-25', '2023-01-25', '2023-01-25']
base = '~/Documents/Molly-thesis/Data/'
rcp='rcp26'

runoff = 'melt_on_glacier'
rcp26_melt_on = to_dict(gcm_list, runoff_stem=runoff, fpath_stem=base, which_date=date_list, which_rcp=rcp)

runoff = 'melt_off_glacier'
rcp26_melt_off = to_dict(gcm_list, runoff_stem=runoff, fpath_stem=base, which_date=date_list, which_rcp=rcp)

runoff = 'liq_prcp_on_glacier'
rcp26_liq_on = to_dict(gcm_list, runoff_stem=runoff, fpath_stem=base, which_date=date_list, which_rcp=rcp)

runoff = 'liq_prcp_off_glacier'
rcp26_liq_off = to_dict(gcm_list, runoff_stem=runoff, fpath_stem=base, which_date=date_list, which_rcp=rcp)

rcp26_melt_on_df = to_df(rcp26_melt_on)
rcp26_melt_off_df = to_df(rcp26_melt_off)
rcp26_liq_on_df = to_df(rcp26_liq_on)
rcp26_liq_off_df = to_df(rcp26_liq_off)

In [74]:
gcm_list = ['CCSM4', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'CanESM2', 'GFDL-CM3', 'GFDL-ESM2G', 'GISS-E2-R', 'IPSL-CM5A-LR', 'MPI-ESM-LR', 'NorESM1-M']
date_list = ['2023-01-12', '2023-01-23', '2023-01-17', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-25', '2023-01-25', '2023-01-25']
base = '~/Documents/Molly-thesis/Data/'
rcp='rcp45'

runoff = 'melt_on_glacier'
rcp45_melt_on = to_dict(gcm_list, runoff_stem=runoff, fpath_stem=base, which_date=date_list, which_rcp=rcp)

runoff = 'melt_off_glacier'
rcp45_melt_off = to_dict(gcm_list, runoff_stem=runoff, fpath_stem=base, which_date=date_list, which_rcp=rcp)

runoff = 'liq_prcp_on_glacier'
rcp45_liq_on = to_dict(gcm_list, runoff_stem=runoff, fpath_stem=base, which_date=date_list, which_rcp=rcp)

runoff = 'liq_prcp_off_glacier'
rcp45_liq_off = to_dict(gcm_list, runoff_stem=runoff, fpath_stem=base, which_date=date_list, which_rcp=rcp)

rcp45_melt_on_df = to_df(rcp45_melt_on)
rcp45_melt_off_df = to_df(rcp45_melt_off)
rcp45_liq_on_df = to_df(rcp45_liq_on)
rcp45_liq_off_df = to_df(rcp45_liq_off)

In [75]:
gcm_list = ['CCSM4', 'CSIRO-Mk3-6-0', 'GFDL-CM3', 'GFDL-ESM2G', 'GISS-E2-R', 'IPSL-CM5A-LR', 'NorESM1-M']
date_list = ['2023-01-12', '2023-01-17', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-25', '2023-01-25']
base = '~/Documents/Molly-thesis/Data/'
rcp='rcp60'

runoff = 'melt_on_glacier'
rcp60_melt_on = to_dict(gcm_list, runoff_stem=runoff, fpath_stem=base, which_date=date_list, which_rcp=rcp)

runoff = 'melt_off_glacier'
rcp60_melt_off = to_dict(gcm_list, runoff_stem=runoff, fpath_stem=base, which_date=date_list, which_rcp=rcp)

runoff = 'liq_prcp_on_glacier'
rcp60_liq_on = to_dict(gcm_list, runoff_stem=runoff, fpath_stem=base, which_date=date_list, which_rcp=rcp)

runoff = 'liq_prcp_off_glacier'
rcp60_liq_off = to_dict(gcm_list, runoff_stem=runoff, fpath_stem=base, which_date=date_list, which_rcp=rcp)

rcp60_melt_on_df = to_df(rcp60_melt_on)
rcp60_melt_off_df = to_df(rcp60_melt_off)
rcp60_liq_on_df = to_df(rcp60_liq_on)
rcp60_liq_off_df = to_df(rcp60_liq_off)

In [76]:
gcm_list = ['CCSM4', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'CanESM2', 'GFDL-CM3', 'GFDL-ESM2G', 'GISS-E2-R', 'IPSL-CM5A-LR', 'MPI-ESM-LR', 'NorESM1-M']
date_list = ['2023-01-12', '2023-01-23', '2023-01-17', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-23', '2023-01-25', '2023-01-25', '2023-01-25']
base = '~/Documents/Molly-thesis/Data/'
rcp='rcp85'

runoff = 'melt_on_glacier'
rcp85_melt_on = to_dict(gcm_list, runoff_stem=runoff, fpath_stem=base, which_date=date_list, which_rcp=rcp)

runoff = 'melt_off_glacier'
rcp85_melt_off = to_dict(gcm_list, runoff_stem=runoff, fpath_stem=base, which_date=date_list, which_rcp=rcp)

runoff = 'liq_prcp_on_glacier'
rcp85_liq_on = to_dict(gcm_list, runoff_stem=runoff, fpath_stem=base, which_date=date_list, which_rcp=rcp)

runoff = 'liq_prcp_off_glacier'
rcp85_liq_off = to_dict(gcm_list, runoff_stem=runoff, fpath_stem=base, which_date=date_list, which_rcp=rcp)

rcp85_melt_on_df = to_df(rcp85_melt_on)
rcp85_melt_off_df = to_df(rcp85_melt_off)
rcp85_liq_on_df = to_df(rcp85_liq_on)
rcp85_liq_off_df = to_df(rcp85_liq_off)

In [77]:
rcp26_melt_on = rcp26_melt_on_df
rcp26_melt_off = rcp26_melt_off_df
rcp26_prcp = (rcp26_liq_on_df + rcp26_liq_off_df)

rcp45_melt_on = rcp45_melt_on_df
rcp45_melt_off = rcp45_melt_off_df
rcp45_prcp = (rcp45_liq_on_df + rcp26_liq_off_df)

rcp60_melt_on = rcp60_melt_on_df
rcp60_melt_off = rcp60_melt_off_df
rcp60_prcp = (rcp60_liq_on_df + rcp60_liq_off_df)

rcp85_melt_on = rcp85_melt_on_df
rcp85_melt_off = rcp85_melt_off_df
rcp85_prcp = (rcp85_liq_on_df + rcp85_liq_off_df)

#### Liquid precip

In [84]:
print(rcp26_prcp.iloc[80]['mean'],
      rcp45_prcp.iloc[80]['mean'],
      rcp60_prcp.iloc[80]['mean'],
      rcp85_prcp.iloc[80]['mean'],
     )

34.812347 34.647404 36.51432 36.083523


In [85]:
rcp60_prcp.iloc[80]['mean'] - rcp26_prcp.iloc[80]['mean']

1.701973

GCM spread - precip

In [86]:
rcp26_prcp.iloc[80].drop('mean').max() - rcp26_prcp.iloc[80].drop('mean').min()

8.744442

In [87]:
rcp45_prcp.iloc[80].drop('mean').max() - rcp45_prcp.iloc[80].drop('mean').min()

8.209923

In [88]:
rcp60_prcp.iloc[80].drop('mean').max() - rcp60_prcp.iloc[80].drop('mean').min()

15.196966

In [89]:
rcp85_prcp.iloc[80].drop('mean').max() - rcp85_prcp.iloc[80].drop('mean').min()

27.01123

#### Melt on glacier

In [90]:
print(rcp26_melt_on.iloc[80]['mean'],
      rcp45_melt_on.iloc[80]['mean'],
      rcp60_melt_on.iloc[80]['mean'],
      rcp85_melt_on.iloc[80]['mean'],
     )

1.3366468 1.130198 1.087637 1.1932846


In [92]:
rcp26_melt_on.iloc[80]['mean']-rcp60_melt_on.iloc[80]['mean']

0.24900985

GCM spread - melt on

In [93]:
rcp26_melt_on.iloc[80].drop('mean').max() - rcp26_melt_on.iloc[80].drop('mean').min()

1.6760937

In [94]:
rcp45_melt_on.iloc[80].drop('mean').max() - rcp45_melt_on.iloc[80].drop('mean').min()

0.6737865

In [95]:
rcp60_melt_on.iloc[80].drop('mean').max() - rcp60_melt_on.iloc[80].drop('mean').min()

1.0543994

In [96]:
rcp85_melt_on.iloc[80].drop('mean').max() - rcp85_melt_on.iloc[80].drop('mean').min()

0.8987329

#### Melt off glacier

In [97]:
print(rcp26_melt_off.iloc[80]['mean'],
      rcp45_melt_off.iloc[80]['mean'],
      rcp60_melt_off.iloc[80]['mean'],
      rcp85_melt_off.iloc[80]['mean'],
     )

1.8405902 0.43998298 0.19159718 0.027971124


In [98]:
rcp26_melt_off.iloc[80]['mean']-rcp85_melt_off.iloc[80]['mean']

1.8126191

GCM spread - melt off

In [99]:
rcp26_melt_off.iloc[80].drop('mean').max() - rcp26_melt_off.iloc[80].drop('mean').min()

3.9634352

In [100]:
rcp45_melt_off.iloc[80].drop('mean').max() - rcp45_melt_off.iloc[80].drop('mean').min()

1.1233696

In [101]:
rcp60_melt_off.iloc[80].drop('mean').max() - rcp60_melt_off.iloc[80].drop('mean').min()

0.6604595

In [102]:
rcp85_melt_off.iloc[80].drop('mean').max() - rcp85_melt_off.iloc[80].drop('mean').min()

0.08638221

#### Pct liquid precip

In [111]:
rcp26_prcp.iloc[80]['mean']/rcp26_df.iloc[80]['mean']*100

91.63656234741211

In [112]:
rcp45_prcp.iloc[80]['mean']/rcp45_df.iloc[80]['mean']*100

92.62850284576416

In [113]:
rcp60_prcp.iloc[80]['mean']/rcp60_df.iloc[80]['mean']*100

96.61521315574646

In [114]:
rcp85_prcp.iloc[80]['mean']/rcp85_df.iloc[80]['mean']*100

96.72626256942749

In [115]:
rcp26_prcp.iloc[0]['mean']/rcp26_df.iloc[0]['mean']*100

62.72662878036499

In [116]:
rcp45_prcp.iloc[0]['mean']/rcp45_df.iloc[0]['mean']*100

63.548219203948975

In [117]:
rcp60_prcp.iloc[0]['mean']/rcp60_df.iloc[0]['mean']*100

63.756972551345825

In [118]:
rcp85_prcp.iloc[0]['mean']/rcp85_df.iloc[0]['mean']*100

62.39039897918701

# <b> Seasonal Calculations

Seasonal calculations are in "Seasonal April" notebook